In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aiml-codebase-dataset/ai_ml_training_dataset.jsonl
/kaggle/input/aiml-codebase-dataset/ai_ml_training_chat.csv
/kaggle/input/aiml-codebase-dataset/ai_ml_raw_dataset.csv


In [2]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

try:
    user_secrets = UserSecretsClient()
    # Using your variable to hold the token
    secret_value_1 = user_secrets.get_secret("HF_TOKEN")
    
    # Passing your variable to the login function
    login(token=secret_value_1)
    
    print("✅ Successfully logged in to Hugging Face Hub using the token.")
except Exception as e:
    print(f"⚠️ Could not log in. Please ensure 'HF_TOKEN' is a valid secret. Error: {e}")

✅ Successfully logged in to Hugging Face Hub using the token.


In [3]:
!pip install -q -U transformers datasets accelerate peft bitsandbytes trl 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 78.3 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 26.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 76.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.1 MB/s eta 0:00:0000:0

In [4]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer , SFTConfig

# 1. Configuration
# Model and tokenizer names
model_name = "google/codegemma-7b-it"

# Dataset path
# 💡 IMPORTANT: Update this to the path where you uploaded your dataset.jsonl file
dataset_path = "/kaggle/input/aiml-codebase-dataset/ai_ml_training_dataset.jsonl" 

# Fine-tuned model name
new_model_name = "CodeGemma-7B-Docstring-Generator"

# 2. Define the Prompt Template Function (UPDATED)
def format_chat_template(sample):
    """
    Formats the sample data from the JSONL file into the CodeGemma chat template.
    This function is specifically designed to handle the {"messages": [...]} structure.
    """
    # Extract user content and assistant content
    user_message = sample['messages'][0]['content']
    assistant_message = sample['messages'][1]['content']
    
    # Apply the CodeGemma chat template
    formatted_prompt = f"<start_of_turn>user\n{user_message}<end_of_turn>\n<start_of_turn>model\n{assistant_message}<end_of_turn>"
    
    return {"text": formatted_prompt}
dataset = load_dataset('json', data_files=dataset_path, split='train')

2025-09-14 21:33:58.131913: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757885638.397031      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757885638.468507      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
dataset = dataset.select(range(1000))

In [6]:
dataset = dataset.map(format_chat_template)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:

# 3. Load Dataset and Apply Template

# 4. Configure Quantization 
# Use 4-bit quantization to fit the model on T4 GPUs
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# 5. Load Base Model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# 6. Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# Set a padding token if one is not already defined
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 7. PEFT/LoRA Configuration
# LoRA enhances training efficiency by updating only a small subset of model parameters
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

output_dir = "/kaggle/working/results"
os.makedirs(output_dir, exist_ok=True)

# 8. Training Arguments
training_arguments = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    warmup_steps=5,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=50,
    learning_rate=2e-4,
    weight_decay=0.001,
    bf16=True,
    optim="paged_adamw_8bit",
    max_length=1024,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant':False},
    report_to="none",
    disable_tqdm=False,
    packing = False,
)

# 9. Initialize the Trainer
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=dataset,
    peft_config=peft_config,
    formatting_func=format_chat_template,
    args=training_arguments,
)

# 10. Start Training
print("🚀 Starting training...")
trainer.train()
print("✅ Training complete!")

# 11. Save the Fine-Tuned Model Adapter
trainer.model.save_pretrained(new_model_name)
tokenizer.save_pretrained(new_model_name)
print(f"✅ Model saved to {new_model_name}")

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

🚀 Starting training...


Step,Training Loss
10,1.319200
20,0.729200
30,0.666900
40,0.626200
50,0.561000
60,0.568500


✅ Training complete!
✅ Model saved to CodeGemma-7B-Docstring-Generator


In [8]:
print("🚀 Pushing model to the Hub...")

# Define your Hugging Face repository ID
# IMPORTANT: Replace "YourHuggingFaceUsername" with your actual username.
repo_id = "Tejas1615/CodeGemma-7B-Docstring-Generator" 

# Push the model adapter. It will create the repo if it doesn't exist.
# Use private=True if you want to keep the model private.
trainer.model.push_to_hub(repo_id, private=False)

# Push the tokenizer
trainer.tokenizer.push_to_hub(repo_id, private=False)

print(f"✅ Model successfully pushed to https://huggingface.co/{repo_id}")
# ---------------------------------------------------------------- #

🚀 Pushing model to the Hub...


Uploading...:   0%|          | 0.00/200M [00:00<?, ?B/s]

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


README.md: 0.00B [00:00, ?B/s]

Uploading...:   0%|          | 0.00/38.6M [00:00<?, ?B/s]

✅ Model successfully pushed to https://huggingface.co/Tejas1615/CodeGemma-7B-Docstring-Generator
